In [1]:
import os
from flood_tool import *
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout
import warnings
warnings.filterwarnings("ignore")

In [2]:
# User Defined Inputs
date='2020-06-10'
geometry=[88.0,25.0,89.1,26]
individual_probability=True # Set this to true if you want individual model probabilities tiff as well
user_path=os.path.expanduser('~/Downloads/SAR_flood')
# user_path=os.path.join('f:',os.sep,'dl_flood')

In [3]:
gpu_check()

1 Physical GPUs,


In [4]:
Initialize()

Earth Engine initialized.


In [5]:
path=createDirectory(os.path.join(user_path,'SAR_flood'))
probability_path=createDirectory(os.path.join(path,'probability_'+'_'.join([str(elem) for elem in geometry])))

C:\Users\nirde/Downloads/SAR_flood\SAR_flood --> Folder exists!!!
------------------Using existing folder!!!-----------------
Folder not found!!!   C:\Users\nirde/Downloads/SAR_flood\SAR_flood\probability_88.0_25.0_89.1_26 created.
------Folder Creation Done!!!--------


In [6]:
sentinel1,dem,slope,jrc=get_files_from_gee(date,geometry,path)
s1_chunked,dem_chunked,slope_chunked,jrc_chunked=Parallel(n_jobs=4,backend='threading')(delayed(chunk_band)(path,file) for file in [sentinel1,dem,slope,jrc])

Folder not found!!!   C:\Users\nirde/Downloads/SAR_flood\SAR_flood\gee_files88.0_25.0_89.1_26 created.
------Folder Creation Done!!!--------
--------------Files download getting ready ---------------------


Consider adjusting `region`, `scale` and/or `dtype` to reduce the s1_88.0_25.0_89.1_26.tif download size (raw: 2.18 GB).


slope_88.0_25.0_89.1_26.tif: |          | 0.00/546M (raw) [  0.0%] in 00:00 (eta:     ?)

s1_88.0_25.0_89.1_26.tif: |          | 0.00/2.18G (raw) [  0.0%] in 00:00 (eta:     ?)

jrc_88.0_25.0_89.1_26.tif: |          | 0.00/136M (raw) [  0.0%] in 00:00 (eta:     ?)

dem_88.0_25.0_89.1_26.tif: |          | 0.00/273M (raw) [  0.0%] in 00:00 (eta:     ?)

In [ ]:
s1_chunked=os.path.join(path,'s1_89_25.0_89.1_25.1')
dem_chunked=os.path.join(path,'dem_89_25.0_89.1_25.1')
slope_chunked=os.path.join(path,'slope_89_25.0_89.1_25.1')
jrc_chunked=os.path.join(path,'jrc_89_25.0_89.1_25.1')

In [ ]:
location_database=get_location_database(jrc_chunked,dem_chunked,slope_chunked,s1_chunked)
dimension_check(location_database)

In [ ]:
display_input_chip(location_database)

In [ ]:
model1=load_model(r'F:\python_nb\mod\unet_tversky_loss_resnext50.h5',compile=False)
model2=load_model(r'F:\python_nb\mod\unet_focal_tversky_loss_resnext50.h5',compile=False)
model3=load_model(r'F:\python_nb\mod\unet_tversky_loss_efficientnetb0.h5',compile=False,custom_objects={'FixedDropout':Dropout})

In [ ]:
create_output_tiff([model1,model2,model3],location_database,probability_path,individual_probability)

In [ ]:
display_output_chip(probability_path)

In [ ]:
create_combined_probability_tiff(probability_path)
split_raster_bands(probability_path)

# Run the next code only if you want to remove all raw files

In [ ]:
# remove_data(path)